In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [4]:
class Net(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(3,16,kernel_size=3,padding=1)
    self.act1 = nn.Tanh()
    self.pool1 = nn.MaxPool2d(2)
    self.conv2 = nn.Conv2d(16,8,kernel_size=3,padding=1)
    self.act2 = nn.Tanh()
    self.pool2 = nn.MaxPool2d(2)
    self.fc1 = nn.Linear(8*8*8,32)
    self.act3 = nn.Tanh()
    self.fc2 = nn.Linear(32,2)

  def forward(self,x):
    out = self.pool1(self.act1(self.conv1(x)))
    out = self.pool2(self.act2(self.conv2(out)))
    out = out.view(-1,8*8*8)
    out = self.act3(self.fc1(out))
    out = self.fc2(out)
    return out

In [5]:
model = Net()

numel_list = [p.numel() for p in model.parameters()]
sum(numel_list),numel_list

(18090, [432, 16, 1152, 8, 16384, 32, 64, 2])

In [7]:
class Net(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(3,16,kernel_size=3,padding=1)
    #self.act1 = nn.Tanh()
    #self.pool1 = nn.MaxPool2d(2)
    self.conv2 = nn.Conv2d(16,8,kernel_size=3,padding=1)
    #self.act2 = nn.Tanh()
    #self.pool2 = nn.MaxPool2d(2)
    self.fc1 = nn.Linear(8*8*8,32)
    #self.act3 = nn.Tanh()
    self.fc2 = nn.Linear(32,2)

  def forward(self,x):
    out = F.max_pool2d(torch.tanh(self.conv1(x)),2)
    out = F.max_pool2d(torch.tanh(self.conv2(out)),2)
    out = out.view(-1,8*8*8)
    out = torch.tanh(self.fc1(out))
    out = self.fc2(out)
    return out

In [13]:
model = Net()

numel_list = [p.numel() for p in model.parameters()]
sum(numel_list),numel_list

(18090, [432, 16, 1152, 8, 16384, 32, 64, 2])

In [8]:
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [9]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))]
)

trainset = torchvision.datasets.CIFAR10(root='./data',train=True,
                                        download=True,transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size = 4,
                                                shuffle=True,num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data',train=False,
                                        download=True,transform=transform)


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [10]:
classes = ('plane','car','bird','cat','deer','dog','frog','horse','ship','truck')

In [11]:
label_map = {0:0,2:1}
class_names = ['airplane','bird']
cifar2 = [(img,label_map[label]) for img,label in trainset if label in [0,2]]
cifar2_val = [(img,label_map[label]) for img,label in testset if label in [0,2]]

In [14]:
img, _ = cifar2[0]

model = Net()
model(img.unsqueeze(0))

tensor([[-0.0107,  0.0774]], grad_fn=<AddmmBackward>)

In [15]:
import datetime
def training_loop(n_epochs,optimizer,model,loss_fn,train_loader):
  for epoch in range(1,n_epochs+1):
    loss_train = 0.0
    for imgs, labels in train_loader:
      outputs = model(imgs)
      loss = loss_fn(outputs,labels)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      loss_train += loss.item()

    if epoch==1 or epoch%10 ==0:
      print('{} Epoch {}, Training loss {}'.format(datetime.datetime.now(),epoch,loss_train/len(train_loader)))

In [16]:
train_loader = torch.utils.data.DataLoader(cifar2,batch_size=64,shuffle=True)

model = Net()
optimizer = optim.SGD(model.parameters(),lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

training_loop(
    n_epochs=100,
    optimizer=optimizer,
    model=model,
    loss_fn=loss_fn,
    train_loader=train_loader
)

2020-10-22 11:14:55.771885 Epoch 1, Training loss 0.6274235210600932
2020-10-22 11:15:38.124416 Epoch 10, Training loss 0.35504152070564826
2020-10-22 11:16:23.639614 Epoch 20, Training loss 0.3125564468324564
2020-10-22 11:17:09.908413 Epoch 30, Training loss 0.2941363675009673
2020-10-22 11:17:55.893935 Epoch 40, Training loss 0.2746715191633079
2020-10-22 11:18:42.224387 Epoch 50, Training loss 0.25673817895400297
2020-10-22 11:19:28.283009 Epoch 60, Training loss 0.24234872814386513
2020-10-22 11:20:14.273413 Epoch 70, Training loss 0.22940956393055095
2020-10-22 11:21:00.300564 Epoch 80, Training loss 0.21562742081227576
2020-10-22 11:21:46.488675 Epoch 90, Training loss 0.19903665382391328
2020-10-22 11:22:33.353764 Epoch 100, Training loss 0.1865999854294358
